In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../train_data/users_data.csv")
df["user_id"] = df["user_id"].apply(lambda x: x-1)
df.head()

,user_id,gender,age,occupation,zip_code
0,0,M,35,17,49508
1,1,M,35,1,10918
2,2,M,25,20,14895
3,3,F,25,0,97401
4,4,M,35,12,75069


In [3]:
users = 3974

genres_map = {
    "Action": 0,
    "Adventure": 1,
    "Animation": 2,
    "Children's": 3,
    "Comedy": 4,
    "Crime": 5,
    "Documentary": 6,
    "Drama": 7,
    "Fantasy": 8,
    "Film-Noir": 9,
    "Horror": 10,
    "Musical": 11,
    "Mystery": 12,
    "Romance": 13,
    "Sci-Fi": 14,
    "Thriller": 15,
    "War": 16,
    "Western": 17,
}

sex_map = {
    "M": 0,
    "F": 1,
}

age_map = {
    1: 0,
    18: 1,
    25: 2,
    35: 3,
    45: 4,
    50: 5,
    56: 6,
}

occupation_map = {
    o: o for o in range(21)
}

In [4]:
def generate_user_encoding(ids, genders, ages, occupations):
        out_sex = np.zeros((users, len(sex_map)), dtype=np.float32)
        out_age = np.zeros((users, len(age_map)), dtype=np.float32)
        out_occupation = np.zeros((users, len(occupation_map)), dtype=np.float32)

        for user_id, gender, age, occupation in zip(ids, genders, ages, occupations):
            out_sex[user_id, sex_map[gender]] = 1
            out_age[user_id, age_map[age]] = 1
            out_occupation[user_id, occupation_map[occupation]] = 1

        return out_sex, out_age, out_occupation

In [5]:
generate_user_encoding(
    df["user_id"].tolist(), 
    df["gender"].tolist(), 
    df["age"].tolist(), 
    df["occupation"].tolist(), 
)

(array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [6]:
import sys
sys.path.append("../")

from torch.utils.data import DataLoader
from utils.dataset import RatingsDatasetAllData

In [7]:
movies = 3564

In [8]:
train_dataloader = DataLoader(
    RatingsDatasetAllData(
        users,
        movies,
        "../train_data/train.csv",
        "../train_data/movies_data.csv",
        "../train_data/users_data.csv",
        "user_id",
        "movie_id",
        "rating",
        "genres",
        genres_map,
        "gender",
        sex_map,
        "age",
        age_map,
        "occupation",
        occupation_map,
    ), 
    batch_size=4,
    num_workers=1,
    shuffle=True,
)

In [13]:
for i_batch, (vus, vis, rs, gs, sxs, ags, ocs) in enumerate(train_dataloader):
    print(gs)
    break

tensor([[1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])
